# Control Systems Tutorial 
## Part B: Control tuning

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import control as ct
import source.eins_controller as controller
from source.cablecar_model import cablecar_ode, cablecar_output

# System parameters 
M = 100; m = 900; l = 5.0; g = 9.81; gammax = 20.0; gammaphi = 2.0

cablecar_params = {
    "M": M,           # mass of the slider
    "m": m,         # mass of the cable car
    "l": l,         # length of the suspension cable
    "g": g,         # gravitational acceleration
    "gammax": gammax,       # damping of the slider
    "gammaphi": gammaphi,       # damping of the cable car
}

cablecar_sys = ct.nlsys(
    cablecar_ode, cablecar_output, name='cablecar',
    params=cablecar_params, states=['phi', 'omega', 'x', 'v'],
    outputs=['phi', 'omega', 'x', 'v'], inputs=['F'])

x0, u0 = [-0.1, 0.0, 0.0, 0.0], [0.0]
xeq, ueq = ct.find_operating_point(cablecar_sys, x0, u0)
cablecar_sys_linear = ct.linearize(cablecar_sys, xeq, ueq) 


## Your overall task

As engineers at the EINS cable car company, your mission is to design a position control system for cable cars.

Your colleagues from Product Engineering have devised the following requirements for the control system:
- Reach the terminal position so that passengers can safely exit.
- Provide a pleasant cable car ride experience. Note: Cable cars are not roller coasters.
- Be efficient in terms of control power and do not lead to excessive wear of the actuator.

Although the product engineers did not specify it, remember that a control system that leads to an unstable system will not pass the testing stage.

### Manual tuning of PID

__Task:__ Manually tune a PID controller to control the position of the cable car.

#### System model 

First, we need to derive the transfer function of the linearized cable car model for the input force to output position.
This means we need to compute

$$ G(s)=c_{position}(sI-A)^{-1}b_{force} .$$

#### Analyze the transfer function:
*Transfer function*

In [ ]:

cable_car_f2x= controller.siso_position(cablecar_sys_linear)
print(cable_car_f2x)


#### Define a PID controller by its transfer function

In [ ]:
pid_sys = controller.pid_tf(1,1,1)
print(pid_sys)


#### Design a first controller

In [ ]:
pid = controller.pid_tf(1,0,0)
closed_sys = controller.close_siso_sys(cable_car_f2x, pid)
y=controller.evaluate_step_response(closed_sys)["y_out"]


In [ ]:
T, e = ct.forced_response(pid, controller.DEFAULT_TIMEPTS, 1 - y)
fig, axs = plt.subplots(2, 1, sharex=True, figsize=(8, 5))

axs[0].plot(T, 1-y, 'b')
axs[0].set_ylabel("Received Error e(t)")
axs[0].grid(True)
axs[0].set_title("PID Controller received Error and Input")
axs[1].plot(T, e, 'r')
axs[1].set_ylabel("Input u(t)")
axs[1].set_xlabel("Time [s]")
axs[1].grid(True)

plt.tight_layout()
plt.show()

*Question:* Which influence does the constant $\tau$ have on the PID controller?

#### First test of PID position control

__Test case:__ Follow the position trajectory $p(t)$

$p(t) = \min(t,20),t\in[0,40]$

__Settling time:__ We define the settling time as $T_{settle} \in \arg \max t, \text{s.t.} \|x_{final} - x_{c,t}\|_1 \ge 0.01m$

In [ ]:
y_out, _ =controller.analyze_pid_control(cable_car_f2x,pid)

*Let us analyze the other states that are not controlled by the PID.*

In [ ]:
controller.analyze_pid_misc_states(pid,cablecar_sys_linear, controller.DEFAULT_POS_TRAJECTORY-y_out)

*Please mind the gap - PID Tuning again*: 
Maybe we indeeed need an $K_I > 0$...

In [ ]:
pid = controller.pid_tf(1,0,0)
y_out, _ =controller.analyze_pid_control(cable_car_f2x,pid)

*Let us analyze the other states that are not controlled by the PID.*


In [ ]:
controller.analyze_pid_misc_states(pid, cablecar_sys_linear, controller.DEFAULT_POS_TRAJECTORY-y_out)

*Have we derived a properly working position control system?*

### Analyzing the PID using root-locus curve

What can we derive from the root locus curve? Will the performance improve if we increase all PID gains equally?

In [ ]:
open_loop = cable_car_f2x*pid


*Poles*

In [ ]:
open_loop.poles()

*Zeros*

In [ ]:
open_loop.zeros()

In [ ]:
ct.rlocus(open_loop)

#### Conclusion

It is difficult to tune the control performance using only pole placement because the relationship between poles and performance (e.g., overshoot and settling time) is not straightforward.

### Deployment of state-feedback controller using LQR

Now, let's try devising a more effective controller using an optimal state-feedback controller. 

#### First we need to check if the system is controllable

In [ ]:
# Controllability matrix
sys = cablecar_sys_linear
Co, rank_Co = controller.check_controllability(sys)

*Conclusion:* Fortunately, the system is controllable. Therefore, we can begin designing our LQR controller. 

#### LQR design

The LQR optimization problem

$$\begin{align*}
\min_{\mathbf{u}} \; & J = \int_0^{\infty} \left( \mathbf{x}^\top \mathbf{Q} \mathbf{x} + \mathbf{u}^\top \mathbf{R} \mathbf{u} \right) \, dt \\
\text{given} \quad & \dot{\mathbf{x}} = \mathbf{A} \mathbf{x} + \mathbf{B} \mathbf{u}
\end{align*}$$

In [ ]:
Q = np.eye(sys.A.shape[0])
R = 1
clsys, K, S, E =controller.compute_lqr(sys, Q, R)
resp_lqr = controller.resp_for_input(clsys)

*Analyze the position control and its input*

In [ ]:
controller.analyze_lqr_control(resp_lqr)


*Analyze the other output states*

In [ ]:
controller.analyze_lqr_misc_states(resp_lqr)

*Are you satisfied with the controller's performance?* If not, change the $Q$ and $R$ matrices!